In [9]:
import pandas as pd

# Load the CSV file
csv_file = 'elevation.csv'
df = pd.read_csv(csv_file)

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Seconds of Week  Solution Status  Number of Observations  PRN  \
0          260         200602.0                0                       8   24   
1          260         200602.0                0                       8   17   
2          260         200602.0                0                       8   22   
3          260         200602.0                0                       8    6   
4          260         200602.0                0                       8   19   

      Azimuth  Elevation  Residual  Reject Code  
0  285.814370  39.748315  1.339807            0  
1   48.899215  15.490788 -3.413657            0  
2   88.200461  44.565553  0.002928            0  
3   22.876740  59.979183  1.468951            0  
4   25.936024  31.261930  0.529552            0  

Tail of the file:
         Week Number  Seconds of Week  Solution Status  \
8810475   1098325777        -0.000883       3248381998   
8810476   1098325777        -0.000883       3248381998   
88

In [10]:
# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Seconds of Week', 'PRN', 'Elevation']
df = df[columns_to_keep]
df = df[df['Seconds of Week'] >= 0]


In [11]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df['UTC Time'] = df.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df['24-hour Time'] = df['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df.head())


Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Elevation            UTC Time  \
0          260         200602.0   24  39.748315 1985-01-01 07:43:22   
1          260         200602.0   17  15.490788 1985-01-01 07:43:22   
2          260         200602.0   22  44.565553 1985-01-01 07:43:22   
3          260         200602.0    6  59.979183 1985-01-01 07:43:22   
4          260         200602.0   19  31.261930 1985-01-01 07:43:22   

  24-hour Time  
0     07:43:22  
1     07:43:22  
2     07:43:22  
3     07:43:22  
4     07:43:22  


In [12]:
import plotly.io as pio
pio.renderers.default = "iframe"
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN
for prn in df['PRN'].unique():
    group = df[df['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['Elevation'], mode='lines', name=f'PRN {prn}'))

# Update layout
fig.update_layout(
    title='d/u values for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='Elevation',
    xaxis_tickformat='%H:%M:%S'
)

# Show the figure
fig.show()


In [13]:
grouped = df.groupby('PRN')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 5
Head of the group:
      Week Number  Seconds of Week  PRN  Elevation            UTC Time  \
1033          260         200858.0    5        0.0 1985-01-01 07:47:38   
1042          260         200859.0    5        0.0 1985-01-01 07:47:39   
1051          260         200860.0    5        0.0 1985-01-01 07:47:40   
1060          260         200861.0    5        0.0 1985-01-01 07:47:41   
1069          260         200862.0    5        0.0 1985-01-01 07:47:42   

     24-hour Time  
1033     07:47:38  
1042     07:47:39  
1051     07:47:40  
1060     07:47:41  
1069     07:47:42  

Tail of the group:
      Week Number  Seconds of Week  PRN  Elevation            UTC Time  \
1204          260         200877.0    5   0.000000 1985-01-01 07:47:57   
1213          260         200878.0    5   0.000000 1985-01-01 07:47:58   
1222          260         200879.0    5   0.000000 1985-01-01 07:47:59   
1231          260         200904.0    5   9.723771 1985-01-01 07:48:24   
1241          260 

In [14]:
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    fig = px.line(group, x='24-hour Time', y='Elevation', title=f'd/u values for PRN {specific_prn}')
    fig.update_layout(
        xaxis_title='Time (24-hour format)',
        yaxis_title='Elevation',
        xaxis_tickformat='%H:%M:%S'
    )
    fig.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  19


In [ ]:
19
